In [1]:
import subprocess
import glob
import os
from os.path import exists
import re
import pandas as pd
from multiprocessing import Pool
import pyreadr

/media/HDD2/donghui/bulk_ATAC_DM1_DM2d/scan_motif » nohup fimo --oc fimo_single_thread Ath_TF_binding_motifs_plantTFDB.meme /media/HDD2/Genomes/Ath_Ensembl56/Arabidopsis_thaliana.TAIR10.dna.toplevel.fa &

In [2]:
from Bio import motifs
import re
from pathlib import Path

# Load the motifs from the MEME file
with open('./scan_motif/Ath_TF_binding_motifs_plantTFDB.meme') as f:
    meme_content = f.read()
    # Splitting the file content by the 'MOTIF' keyword, but keeping the 'MOTIF' part
    motifs_blocks = re.split(r'(?=MOTIF)', meme_content)[1:]  # Ignore the header before the first 'MOTIF'

# Directory where the individual motif files will be saved
output_dir = Path('./scan_motif/individual_motifs')
output_dir.mkdir(parents=True, exist_ok=True)

# Write each motif to a separate file named after the motif identifier
for motif_block in motifs_blocks:
    motif_id = re.search(r'MOTIF\s+(\w+)', motif_block).group(1)
    motif_file = output_dir / f'{motif_id}.meme'
    with open(motif_file, 'w') as f:
        # Write the motif block to file
        f.write(meme_content[:meme_content.index('MOTIF')] + motif_block.strip())  # Include header and motif

print(f'{len(motifs_blocks)} motifs were split into individual files.')


619 motifs were split into individual files.


In [3]:
import os
import subprocess
from multiprocessing import Pool
from pathlib import Path

# Function to run FIMO for a given motif file
def run_fimo(motif_file):
    # Extract the motif identifier from the file name
    motif_id = motif_file.stem
    # Define the output directory for this motif's FIMO results
    output_dir = Path('./scan_motif/individual_fimo_scan') / motif_id
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Define the log file path
    log_file_path = output_dir / 'fimo.log'
    
    # Open the log file
    with open(log_file_path, 'w') as log_file:
        # Build the FIMO command
        cmd = [
            'fimo',
            '--oc', str(output_dir),
            str(motif_file),
            '/media/HDD2/Genomes/Ath_Ensembl56/Arabidopsis_thaliana.TAIR10.dna.toplevel.fa'
        ]
        
        # Run the command and redirect stdout and stderr to the log file
        subprocess.run(cmd, stdout=log_file, stderr=subprocess.STDOUT, check=True)

# Directory where the individual motif files are saved
motif_dir = Path('./scan_motif/individual_motifs')
motif_files = list(motif_dir.glob('*.meme'))

# Set up the pool of workers and run FIMO on each motif file
if __name__ == '__main__':
    with Pool(processes=120) as pool:  # Adjust the number of processes as needed
        pool.map(run_fimo, motif_files)


In [5]:
! rm -rf ./scan_motif/PWMscan_fimo_ath

In [26]:
import os
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor

# Function to convert a TSV file to a BED file
def convert_tsv_to_bed(tsv_file, output_dir):
    bed_file_name = tsv_file.parent.name + '_TFBS.bed'
    output_file_path = output_dir / bed_file_name
    
    lines_written = False  # Flag to check if any line is written to the file

    with open(tsv_file, 'r') as f:
        next(f)  # Skip the header line
        lines = []
        for line in f:
            parts = line.strip().split('\t')
            if len(parts) < 10:
                continue  # Skip lines that don't have enough columns
            chrom = f'{parts[2]}'
            start = str(int(parts[3]) - 1)  # Convert to 0-based start position
            end = parts[4]
            # name = parts[0]
            matched_seq = parts[9]
            score = parts[6]
            strand = parts[5]
            bed_line = f'{chrom}\t{start}\t{end}\t{matched_seq}\t{score}\t{strand}\n'
            lines.append(bed_line)
            lines_written = True
    
    # Write to the output file only if there are lines to write
    if lines_written:
        with open(output_file_path, 'w') as out_f:
            for line in lines:
                out_f.write(line)
    else:
        print(f'No valid entries for {tsv_file}. No BED file created.')

# Set the base directories
fimo_scan_dir = Path('/media/HDD2/donghui/bulk_ATAC_DM1_DM2d/scan_motif/individual_fimo_scan')
output_dir = Path('./scan_motif/PWMscan_fimo_ath')
output_dir.mkdir(parents=True, exist_ok=True)

# Glob all the fimo.tsv files
tsv_files = list(fimo_scan_dir.glob('**/fimo.tsv'))

# Process the TSV files in parallel
with ThreadPoolExecutor(10) as executor:
    # Using a lambda to pass the 'output_dir' to the function
    executor.map(lambda tsv_file: convert_tsv_to_bed(tsv_file, output_dir), tsv_files)

print('Conversion to BED format completed.')


No valid entries for /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/scan_motif/individual_fimo_scan/AT5G25475/fimo.tsv. No BED file created.
No valid entries for /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/scan_motif/individual_fimo_scan/AT5G48670/fimo.tsv. No BED file created.
No valid entries for /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/scan_motif/individual_fimo_scan/AT5G58620/fimo.tsv. No BED file created.
No valid entries for /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/scan_motif/individual_fimo_scan/AT1G60920/fimo.tsv. No BED file created.
Conversion to BED format completed.


In [27]:
! head ./scan_motif/PWMscan_fimo_ath/AT5G62380_TFBS.bed

1	26000638	26000659	TTGCTTGTGTCTCACGTTACC	23.8333	-
4	2776284	2776305	TTGCTTGCTCTTCACGCTACC	23.7576	-
5	19151535	19151556	TTGCTTGTTGATCACGTTACC	23	-
1	22529499	22529520	TTGCTTGGGTTTCAAGCAACC	22.8939	+
3	18636363	18636384	TTGCTTGTTATACAAGTTACC	22.4091	+
5	15789957	15789978	TTGCTTGCTTCTTACGCTTCC	22.3485	-
5	15792242	15792263	TTGCTTGCTTCTTACGCTTCC	22.3485	-
5	15803294	15803315	TTGCTTGCTTCTTACGCTTCC	22.3485	-
5	23372567	23372588	TTGCTTGTGTTGTACGTAACA	22.3485	-
4	8583979	8584000	TTGCTTGTTGTTTACGTTTTC	22.303	-


In [28]:
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor

# Function to add 'chr' prefix to the chromosome name in the BED files
def add_chr_prefix_to_bed(bed_file, input_dir, output_dir):
    modified_lines = []  # List to store modified lines
    with open(input_dir / bed_file, 'r') as f:
        for line in f:
            parts = line.strip().split('\t')
            parts[0] = 'chr' + parts[0]  # Add 'chr' prefix to the chromosome name
            modified_lines.append('\t'.join(parts) + '\n')
    
    # Write the modified lines to the new file in the output directory
    with open(output_dir / bed_file, 'w') as out_f:
        for line in modified_lines:
            out_f.write(line)

# Set the base directories
input_dir = Path('./scan_motif/PWMscan_fimo_ath')
output_dir = Path('/media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/PWMscan_fimo_ath_modified')
output_dir.mkdir(parents=True, exist_ok=True)

# Get all the BED files from the input directory
bed_files = list(input_dir.glob('*.bed'))

# Process the BED files in parallel to add 'chr' prefix
with ThreadPoolExecutor() as executor:
    # Using a lambda to pass both 'input_dir' and 'output_dir' to the function
    executor.map(lambda bed_file: add_chr_prefix_to_bed(bed_file.name, input_dir, output_dir), bed_files)

print('Modification of BED files completed.')


Modification of BED files completed.


In [5]:
! rm -rf ./scan_motif/PWMscan_fimo_ath

## Modify narrowPeak

In [15]:
import os
import pandas as pd
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor
import glob

def process_narrowPeak_file(file_path, output_dir):
    # Read the narrowPeak file using pandas
    df = pd.read_csv(file_path, sep='\t', header=None)
    
    # Modify the first column by adding 'chr' prefix
    df[0] = 'chr' + df[0].astype(str)
    
    # Modify the fourth column by removing the directory path
    df[3] = df[3].apply(lambda x: os.path.basename(x))
    
    # Construct the output file path
    output_file_path = os.path.join(output_dir, os.path.basename(file_path))
    
    # Write the modified DataFrame to the new file
    df.to_csv(output_file_path, sep='\t', header=False, index=False)

# Set the directories
input_dir = Path('/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/5_macs2peaks_simple')
output_dir = '/media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/data/5_macs2peaks_modified'
os.makedirs(output_dir, exist_ok=True)  # Ensure the output directory exists

# Glob all the narrowPeak files
narrowPeak_files = glob.glob(str(input_dir / '*.narrowPeak'))

# Process the narrowPeak files in parallel
with ThreadPoolExecutor(max_workers = 20) as executor:
    for file_path in narrowPeak_files:
        executor.submit(process_narrowPeak_file, file_path, output_dir)

print('Modification of narrowPeak files completed.')


Modification of narrowPeak files completed.


In [29]:
import os
import pandas as pd
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor
import glob

def process_narrowPeak_file(file_path, output_dir):
    # Read the narrowPeak file using pandas
    df = pd.read_csv(file_path, sep='\t', header=None)
    
    # Modify the first column by adding 'chr' prefix
    df[0] = 'chr' + df[0].astype(str)
    
    # Modify the fourth column by removing the directory path
    df[3] = df[3].apply(lambda x: os.path.basename(x))
    
    # Construct the output file path
    output_file_path = os.path.join(output_dir, os.path.basename(file_path))
    
    # Write the modified DataFrame to the new file
    df.to_csv(output_file_path, sep='\t', header=False, index=False)

# Set the directories, DM6-DM7
input_dir = Path('/media/HDD3/bulk_ATAC_DMs/N2315964_30-927980316_Lib_2023-09-30/Analysi_dir/5_macs2peaks_simple')
output_dir = '/media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/data/5_macs2peaks_modified'
os.makedirs(output_dir, exist_ok=True)  # Ensure the output directory exists

# Glob all the narrowPeak files
narrowPeak_files = glob.glob(str(input_dir / '*.narrowPeak'))

# Process the narrowPeak files in parallel
with ThreadPoolExecutor(max_workers = 20) as executor:
    for file_path in narrowPeak_files:
        executor.submit(process_narrowPeak_file, file_path, output_dir)

print('Modification of narrowPeak files completed.')


Modification of narrowPeak files completed.


## the TF table

In [35]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('../gene_table_and_other_scripts/bioMartR_Ath_all_genes_info_googled.csv')

# Rename columns
df = df.rename(columns={'ensembl_gene_id': 'ENSEMBL', 'external_gene_name': 'SYMBOL'})
df['SYMBOL'] = df['SYMBOL'].fillna(df['ENSEMBL'])
# Create new column HOCOID same as SYMBOL
df['HOCOID'] = df['SYMBOL']

# Order the columns as SYMBOL, ENSEMBL, HOCOID
df = df[['SYMBOL', 'ENSEMBL', 'HOCOID']]
df = df.sort_values('ENSEMBL')
df = df.reset_index(drop=True)
# Save the DataFrame to a new CSV file without index
df.to_csv('/media/HDD2/donghui/diffTF_ath/diffTF/src/TF_Gene_TranslationTables/df_translationTable_ath.csv', 
          sep='\t', index=False,)


In [37]:
#change dir to /media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/referenceGenome
os.chdir('/media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/referenceGenome')

In [41]:
##list the files in the directory
os.listdir('./')

['Arabidopsis_thaliana.TAIR10.dna.toplevel.fa']

In [ ]:
# add chr prefix to each chromosome

In [42]:
! sed 's/>\(.*\)/>chr\1/' Arabidopsis_thaliana.TAIR10.dna.toplevel.fa > Arabidopsis_thaliana.TAIR10.modified.fa

In [44]:
import pandas as pd
import os
from multiprocessing import Pool

# Function to modify the chromosome names
def modify_chromosome_names(file_path, target_dir):
    # Read the file
    df = pd.read_csv(file_path, sep='\t', header=None)

    # Modify the first column, assuming it contains the chromosome names
    df[0] = 'chr' + df[0].astype(str)

    # Create a new file path
    new_file_path = os.path.join(target_dir, os.path.basename(file_path))

    # Write the modified DataFrame to the new file path
    df.to_csv(new_file_path, sep='\t', header=False, index=False)

def main():
    # Define the source and target directories
    source_dir = '/media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/PWMscan_fimo_ath'
    target_dir = '/media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/PWMscan_fimo_ath_modified'

    # Create the target directory if it does not exist
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    # Get all .bed files in the source directory
    files = [os.path.join(source_dir, f) for f in os.listdir(source_dir) if f.endswith('.bed')]

    # Use a Pool to process files in parallel
    with Pool(10) as pool:
        pool.starmap(modify_chromosome_names, [(file, target_dir) for file in files])

if __name__ == '__main__':
    main()


EmptyDataError: No columns to parse from file

In [24]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('../gene_table_and_other_scripts/bioMartR_Ath_all_genes_info_googled.csv')

# Rename columns
df = df.rename(columns={'ensembl_gene_id': 'ENSEMBL', 'external_gene_name': 'SYMBOL'})
df['SYMBOL'] = df['SYMBOL'].str.replace(' ', '_')
df['SYMBOL'] = df['SYMBOL'].str.replace('"', '')
df['SYMBOL'] = df['SYMBOL'].fillna(df['ENSEMBL'])
# Create new column HOCOID same as SYMBOL
df['HOCOID'] = df['ENSEMBL']


# Order the columns as SYMBOL, ENSEMBL, HOCOID
df = df[['SYMBOL', 'ENSEMBL', 'HOCOID']]
df = df.sort_values('ENSEMBL')
df = df.reset_index(drop=True)
# Save the DataFrame to a new CSV file without index
print(df.head())
df.to_csv('/media/HDD2/donghui/diffTF_ath/diffTF/src/TF_Gene_TranslationTables/df_translationTable_ath2.csv', 
          sep=' ', index=False,)


       SYMBOL    ENSEMBL     HOCOID
0      NAC001  AT1G01010  AT1G01010
1        ARV1  AT1G01020  AT1G01020
2        NGA3  AT1G01030  AT1G01030
3        DCL1  AT1G01040  AT1G01040
4  ath-MIR838  AT1G01046  AT1G01046


In [37]:
#change dir to /media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/referenceGenome
os.chdir('/media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/referenceGenome')

In [41]:
##list the files in the directory
os.listdir('./')

['Arabidopsis_thaliana.TAIR10.dna.toplevel.fa']

In [ ]:
# add chr prefix to each chromosome

In [42]:
! sed 's/>\(.*\)/>chr\1/' Arabidopsis_thaliana.TAIR10.dna.toplevel.fa > Arabidopsis_thaliana.TAIR10.modified.fa

In [44]:
import pandas as pd
import os
from multiprocessing import Pool

# Function to modify the chromosome names
def modify_chromosome_names(file_path, target_dir):
    # Read the file
    df = pd.read_csv(file_path, sep='\t', header=None)

    # Modify the first column, assuming it contains the chromosome names
    df[0] = 'chr' + df[0].astype(str)

    # Create a new file path
    new_file_path = os.path.join(target_dir, os.path.basename(file_path))

    # Write the modified DataFrame to the new file path
    df.to_csv(new_file_path, sep='\t', header=False, index=False)

def main():
    # Define the source and target directories
    source_dir = '/media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/PWMscan_fimo_ath'
    target_dir = '/media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/PWMscan_fimo_ath_modified'

    # Create the target directory if it does not exist
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    # Get all .bed files in the source directory
    files = [os.path.join(source_dir, f) for f in os.listdir(source_dir) if f.endswith('.bed')]

    # Use a Pool to process files in parallel
    with Pool(10) as pool:
        pool.starmap(modify_chromosome_names, [(file, target_dir) for file in files])

if __name__ == '__main__':
    main()


EmptyDataError: No columns to parse from file